In [2]:
import os

import numpy as np

### Gather all .h5 files

In [7]:
dirpath = '/raid/antoloui/Master-thesis/Data/Embeddings/test/'

# Get the paths of the files.
filepaths = glob.glob(dirpath + '*.h5')

# Create dataframe.
cols = ['feat'+str(i+1) for i in range(768)]
cols.append('Chunk')
df = pd.DataFrame(columns=cols)

# Concat dataframes from all files.
for file in filepaths:
    df_file = pd.read_hdf(file)
    df = pd.concat([df, df_file], ignore_index=True, sort=False)
df

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,...,feat760,feat761,feat762,feat763,feat764,feat765,feat766,feat767,feat768,Chunk
0,0.134384,-0.076830,0.012314,0.101201,-0.049889,-0.086881,-0.036434,0.033909,0.049303,0.012221,...,-0.056202,-0.214364,-0.173924,-0.053610,-0.319917,0.014687,0.178109,0.171774,-0.105795,The Host / Cluster window appears (see Figure ...
1,0.132104,-0.029176,0.076327,0.104246,-0.083129,-0.047523,-0.068164,0.079443,0.046902,-0.013729,...,-0.002110,-0.249440,-0.224281,0.009917,-0.303531,-0.001352,0.196901,0.104056,-0.054210,"To deploy an OVA with the VMware OVF Tool, use..."
2,0.029222,-0.114876,-0.001470,0.044030,0.000200,-0.108047,-0.028566,0.023340,0.115774,0.084451,...,-0.067876,-0.035715,-0.131741,0.059098,-0.270236,0.062539,0.264252,0.146249,0.057102,"After the virtual machine successfully starts,..."
3,0.049390,-0.051062,-0.047536,0.083663,-0.026010,-0.105768,0.016209,-0.006431,0.138516,0.036231,...,-0.110970,-0.140069,-0.085296,0.055840,-0.302315,0.029284,0.303426,0.171609,0.070153,It cannot include the username or default Cisc...
4,0.185348,-0.025731,0.002101,0.085737,-0.079930,-0.032341,-0.017010,-0.026352,0.241939,0.030763,...,-0.141251,-0.189541,-0.142307,0.050496,-0.392716,0.053844,0.250109,0.167576,-0.043903,Note You can change the ANM web server configu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187658,0.154249,-0.189508,-0.261565,-0.006648,0.273737,0.067946,0.001362,0.178214,-0.011738,0.146915,...,-0.067010,-0.024741,-0.335228,0.293198,-0.039774,0.210152,0.056926,-0.157016,0.047978,Analytics > Content > Analyze by Content Title...
187659,0.170222,-0.197636,-0.236919,-0.011768,0.219863,0.090550,0.003200,0.269950,-0.016223,0.157850,...,0.015954,0.047960,-0.352117,0.194707,-0.090628,0.140088,0.014873,-0.124898,-0.009648,Analytics > Content > Analyze by Content Title...
187660,0.166024,-0.060739,0.070918,0.240346,0.101907,-0.024361,-0.086747,-0.036415,0.089530,0.053701,...,-0.152727,-0.186874,-0.212842,-0.038295,-0.197507,0.016903,0.179796,0.075572,0.021567,This chapter includes the following sections: ...
187661,0.145382,-0.022614,0.064664,0.083907,-0.019295,-0.093640,-0.065152,-0.019360,0.086928,-0.026876,...,-0.099399,-0.270253,-0.251326,0.064045,-0.212346,0.141445,0.225926,0.174526,-0.058374,IE 10.0 on Windows 8 Firefox 20 on Windows XP ...


In [ ]:

df